

##***Get the Emoji Package***

In [2]:
pip install emoji

     |████████████████████████████████| 51kB 2.7MB/s 
  Created wheel for emoji: filename=emoji-0.5.4-cp36-none-any.whl size=42176 sha256=399e3f6557c878f5b742b7dc8400a3fc8fb8af50d4320d7426040ae32ad4a001
  Stored in directory: /root/.cache/pip/wheels/2a/a9/0a/4f8e8cce8074232aba240caca3fade315bb49fac68808d1a9c
Successfully built emoji


In [0]:
import emoji

In [0]:
emoji.EMOJI_UNICODE

In [0]:
emoji_dict= {
        '0' : '\u2764\uFE0F',
        '1' : ':baseball:',
        '2' : ':grinning_face_with_big_eyes:',
        '3' : ':disappointed_face:',
        '4' : ':fork_and_knife:'
}

In [31]:
for i in emoji_dict.values():
    print(emoji.emojize(i))

❤️
⚾
😃
😞
🍴


##Processing a Custom dataset

In [0]:
import pandas as pd
import numpy as np

train = pd.read_csv('train_emoji.csv',header=None)
test = pd.read_csv('test_emoji.csv',header=None)

In [0]:
x_train= train.iloc[:,0].values
x_test= test.iloc[:,0].values

y_test = test.iloc[:,1].values
y_train= train.iloc[:,1].values


In [34]:
for i in range(5):
    print(x_train[i],emoji.emojize(emoji_dict[str(y_train[i])]))

never talk to me again 😞
I am proud of your achievements 😃
It is the worst day in my life 😞
Miss you so much ❤️
food is life 🍴


In [0]:
from keras.utils import to_categorical
y_train = to_categorical(y_train,num_classes=5)
y_test = to_categorical(y_test,num_classes=5)

## Embedding the Sentences

In [0]:
f = open('glove.6B.50d.txt',encoding='utf-8')

In [0]:

gloVe_coeff={}
c=1
for line in f:
    values= line.split()
    word = values[0]
    coeff = np.asarray(values[1:],dtype='float')
    gloVe_coeff[word]= coeff
f.close()

In [37]:
gloVe_coeff['eat'].shape

(50,)

##Converting Sentences into Vector

In [0]:
def embedding_output(X):
    maxLen=10
    emb_dim= 50
    embedding_out = np.zeros((X.shape[0],maxLen,emb_dim))
    for i in range(X.shape[0]):
        X[i] = X[i].split()
        for j in range(len(X[i])):
            try:
                embedding_out[i][j] = gloVe_coeff[X[i][j].lower()]
            except:
                embedding_out[i][j] = np.zeros((emb_dim,))
    return embedding_out

['It', 'is', 'the', 'worst', 'day', 'in', 'my', 'life']

In [0]:
embedding_matrix_train = embedding_output(x_train)
embedding_matrix_test = embedding_output(x_test)

In [51]:
print(embedding_matrix_test.shape)
print(embedding_matrix_train.shape)

(56, 10, 50)
(132, 10, 50)


## Building the LSTM

In [0]:
from keras.models import Sequential
from keras.layers import *

classifier =  Sequential()
classifier.add(LSTM(64,input_shape=(10,50),return_sequences=True))
classifier.add(Dropout(rate=0.5))

classifier.add(LSTM(64,input_shape=(10,50)))
classifier.add(Dense(5))
classifier.add(Activation('softmax'))
classifier.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
classifier.summary()

##Training the model


In [61]:
from keras.callbacks import EarlyStopping,ModelCheckpoint

checkpoint = ModelCheckpoint('best_model.h5',monitor='val_loss',verbose=True,save_best_only=True)
stop = EarlyStopping(monitor='val_acc',patience=20)

hist=classifier.fit(embedding_matrix_train,y_train,epochs=200,validation_split=0.2,batch_size=64,shuffle=True,callbacks=[checkpoint,stop])

Train on 105 samples, validate on 27 samples
Epoch 1/200
105/105 [==============================] - 0s 649us/step - loss: 0.4634 - acc: 0.8857 - val_loss: 1.1777 - val_acc: 0.6667

Epoch 00001: val_loss improved from inf to 1.17772, saving model to best_model.h5
Epoch 2/200
105/105 [==============================] - 0s 462us/step - loss: 0.4579 - acc: 0.8667 - val_loss: 1.1188 - val_acc: 0.6296

Epoch 00002: val_loss improved from 1.17772 to 1.11876, saving model to best_model.h5
Epoch 3/200
105/105 [==============================] - 0s 472us/step - loss: 0.4131 - acc: 0.8857 - val_loss: 1.0475 - val_acc: 0.6667

Epoch 00003: val_loss improved from 1.11876 to 1.04752, saving model to best_model.h5
Epoch 4/200
105/105 [==============================] - 0s 481us/step - loss: 0.4143 - acc: 0.8667 - val_loss: 1.0115 - val_acc: 0.7037

Epoch 00004: val_loss improved from 1.04752 to 1.01150, saving model to best_model.h5
Epoch 5/200
105/105 [==============================] - 0s 549us/step - 

In [0]:
classifier.load_weights('best_model.h5')

In [63]:
classifier.evaluate(embedding_matrix_test,y_test)


56/56 [==============================] - 0s 312us/step


[1.2734816244670324, 0.6071428656578064]

In [0]:
pred=classifier.predict_classes(embedding_matrix_test)

In [70]:
for i in range(30):
    print(' '.join(x_test[i]))
    print(emoji.emojize(emoji_dict[str(np.argmax(y_test[i]))]))
    print(emoji.emojize(emoji_dict[str(pred[i])]))

I want to eat
🍴
🍴
he did not answer
😞
😞
he got a raise
😃
😞
she got me a present
❤️
😃
ha ha ha it was so funny
😃
😃
he is a good friend
❤️
😃
I am upset
❤️
😞
We had such a lovely dinner tonight
❤️
😃
where is the food
🍴
🍴
Stop making this joke ha ha ha
😃
😃
where is the ball
⚾
⚾
work is hard
😞
😃
This girl is messing with me
😞
❤️
are you serious ha ha
😃
😞
Let us go play baseball
⚾
⚾
This stupid grader is not working
😞
😞
work is horrible
😞
😞
Congratulation for having a baby
😃
😞
stop messing around
😞
😞
any suggestions for dinner
🍴
🍴
I love taking breaks
❤️
❤️
you brighten my day
😃
❤️
I boiled rice
🍴
🍴
she is a bully
😞
❤️
Why are you feeling bad
😞
😞
I am upset
😞
😞
I worked during my birthday
😞
😃
My grandmother is the love of my life
❤️
❤️
enjoy your break
😃
🍴
valentine day is near
❤️
😃
